In [133]:
# Python
from datetime import datetime

# MPL
import matplotlib        as mpl
import matplotlib.pyplot as plt

# NumPy
import numpy as np
from numpy import cos, exp, log, pi, sin, sqrt
from numpy.random import default_rng

# Scipy
import scipy.linalg as linalg
from scipy.fft import*

In [134]:
# input initial conditions:
L = 1 # m
Rs = L/4 # m
Np = 1 # # of particles
m_p = 1 # kg
part = np.zeros((Np, 9)) # position * 3 (0-2); acceleration * 3 (3-5); velocity * 3 (6-8)
pos = np.zeros((Np, 3)) # position
rng = np.random.default_rng(seed=42) # seed
N_c = 128
N_p = 1
l = L/N_c
G = 6.67430e-11

# Generate random numbers for spherical coordinates:
r = Rs * np.cbrt(rng.uniform(0, 1, Np))
phi = rng.uniform(0, 2 * np.pi, Np)
theta = np.arccos(1 - 2 * rng.uniform(0, 1, Np))

# Convert the spherical coordinates to Cartesian:
x = r * np.sin(theta) * np.cos(phi) + L/2
y = r * np.sin(theta) * np.sin(phi) + L/2 
z = r * np.cos(theta) + L/2

# Add position coordinates: 
pos[:, 0], pos[:, 1], pos[:, 2] = x, y, z

# Print something:
print('The initial conditions is setted.')

The initial conditions is setted.


In [135]:
# def density(part):
#     density = np.zeros((N_c,N_c,N_c))

#     # Assign particles to density array
#     for i in range(N_p):
#         x = part[i,0]
#         y = part[i,1]
#         z = part[i,2]

#         # cells that overlap with the particle
#         i1 = int(np.floor(x/l))
#         i2 = i1+1
#         j1 = int(np.floor(y/l))
#         j2 = j1+1
#         k1 = int(np.floor(z/l))
#         k2 = k1+1

#         # amount of overlap in each dimension
#         delta_x1 = x-i1*l
#         delta_x2 = (i1+1)*l-x
#         delta_y1 = y-j1*l
#         delta_y2 = (j1+1)*l-y
#         delta_z1 = z-k1*l
#         delta_z2 = (k1+1)*l-z

#         # density
#         density[i1,j1,k1] += m_p*delta_x1*delta_y1*delta_z1/(l**3)**2
#         density[i1,j1,k2] += m_p*delta_x1*delta_y1*delta_z2/(l**3)**2
#         density[i1,j2,k1] += m_p*delta_x1*delta_y2*delta_z1/(l**3)**2
#         density[i1,j2,k2] += m_p*delta_x1*delta_y2*delta_z2/(l**3)**2
#         density[i2,j1,k1] += m_p*delta_x2*delta_y1*delta_z1/(l**3)**2
#         density[i2,j1,k2] += m_p*delta_x2*delta_y1*delta_z2/(l**3)**2
#         density[i2,j2,k1] += m_p*delta_x2*delta_y2*delta_z1/(l**3)**2
#         density[i2,j2,k2] += m_p*delta_x2*delta_y2*delta_z2/(l**3)**2
        
#     return density

# # Calculate the total mass in the density array
# density_total_mass = np.sum(density(part))*(l**3)

# # Validate against theoretical total mass
# total_mass = N_p*m_p

# print('mass from density=',density_total_mass)
# print('expected total mass=',total_mass)

In [ ]:
def potential(density):
    # FFT for density
    density_fft = np.fft.rfftn(density)

    # FFT for potential
    potential_fft = np.zeros((N_c,N_c,N_c//2+1),dtype=complex)
    for k_i in range(N_c):
        for k_j in range(N_c):
            for k_k in range(N_c//2+1):
                if k_i <= N_c/2:
                    k_x = 2*pi/N_c*k_i
                else:
                    k_x = 2*pi*(k_i-N_c)/N_c
                if k_j <= N_c/2:
                    k_y = 2*pi/N_c*k_j
                else:
                    k_y = 2*pi*(k_j-N_c)/N_c
                if k_k <= N_c/2:
                    k_z = 2*pi/N_c*k_k
                else:
                    k_z = 2*pi*(k_k-N_c)/N_c
                w_k = -(4*pi*G)/((2*sin(k_x/2))**2+(2*sin(k_y/2))**2+(2*sin(k_z/2))**2+1e-9)
                potential_fft[k_i,k_j,k_k] = density_fft[k_i,k_j,k_k]*w_k*(L/N_c)

    # potential array
    potential = np.fft.irfftn(potential_fft)
    print(potential.shape)
    
    return potential

test = potential(density(pos))
print(test.shape)
test_xy = np.zeros((N_c,N_c))
for i in range(N_c):
    for j in range(N_c):
        test_xy[i,j] = test[i,j,0]

# plt.figure()
# plt.imshow(test_xy,origin='lower',extent=[0.0,1.0,0.0,1.0])
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('potential on x-y plane')
# cb = plt.colorbar()
# plt.show()

In [137]:
def density_field(pos):
    density = np.zeros((N_c, N_c, N_c))

    for part in range(Np):
        x, y, z = pos[part,0], pos[part,1], pos[part,2]

        # grids that contain particles:
        i, j, k = int(np.floor(x / l)), int(np.floor(y / l)), int(np.floor(z / l))

        # Number of overlaps:
        dx, dy, dz = (x / l) - i, (y / l) - j, (z / l) - k

        # Weight:
        weights = [(1 - dx) * (1 - dy) * (1 - dz), dx * (1 - dy) * (1 - dz), (1 - dx) * dy * (1 - dz), dx * dy * (1 - dz), 
            (1 - dx) * (1 - dy) * dz, dx * (1 - dy) * dz, (1 - dx) * dy * dz, dx * dy * dz] # 8 in total
        
        # Update density grid:
        for di, dj, dk, w in zip(
            [0, 1, 0, 1, 0, 1, 0, 1],  # x direction
            [0, 0, 1, 1, 0, 0, 1, 1],  # y direction
            [0, 0, 0, 0, 1, 1, 1, 1],  # z direction
            weights
        ):
            density[(i + di) % N_c, (j + dj) % N_c, (k + dk) % N_c] += m_p * w

    # Normalization:
    density /= l**3

    return density

# check 
density = density_field(pos)
mass = density.sum() * l**3
if int(mass - Np * m_p) == 0:
    print("The density is correct.")
else:
    print("Wrong!")

The density is correct.


In [142]:
# Input data:
G = 6.67430e-11  # m^3 kg^-1 s^-2
# Nc = N_c 
# Nk = Nc
# k = np.arange(Nk)
# k_x = np.zeros(Nk)

# # Wavevector component for x, y:
# for i in range(int(Nk/2 + 1)):
#     k_x[i] = 2 * np.pi / Nc * k[i]

# for j in range(int(Nk/2 + 1), Nk):
#     k_x[j] = 2 * np.pi / Nc * (k[j] - Nk)

# # Wavevector component for z (Due to the rfft transformation):
# k_z = np.zeros(Nk)

# for i in range(Nk // 2 + 1):
#     k_z[i] = 2 * np.pi / Nc * i 
# for j in range(int(Nk/2 + 1), Nk):
#     k_z[j] = 0.0

# Define for kx, ky, and kz:
# kx, ky, kz = np.meshgrid(k_x, k_x, k_z, indexing='ij')

def potential_field(density):
    # FFT for the density field:
    rho_k = np.fft.rfftn(density)

    Nc = N_c 
    Nk = Nc
    k = np.arange(Nk)
    k_x = np.zeros(Nk)
    k_z = np.zeros(Nk)

    # Wavevector component for x, y:
    for i in range(Nk):
        if i < Nk // 2 + 1:
            k_x[i] = 2 * np.pi / Nc * k[i]
            k_z[i] = 2 * np.pi / Nc * i
        else:
            k_x[i] = 2 * np.pi / Nc * (k[i] - Nk)

 # Calculate the gravitational kernel:
        D = ((2 * np.sin(k_x/2))**2 + (2 * np.sin(k_x/2))**2 + (2 * np.sin(k_z/2))**2 + 1e-19) # pull out the denominator
    
    factor = L / Nc # factor due to the normalization of FFt
    omega_k = - (4 * np.pi * G) / D * factor

    # Inverse FFT for the potential field:
    phi = np.fft.irfftn(omega_k * rho_k)

    return phi

In [139]:
# rho_k = np.fft.rfftn(phi)

# kk_z = np.zeros(N_c//2+1)
# kk_x = np.zeros(N_c)
# kk_y = np.zeros(N_c)
# for k_i in range(N_c):
#     for k_j in range(N_c):
#         for k_k in range(N_c//2+1):
#             if k_i <= N_c/2:
#                 k_x = 2*pi/N_c*k_i
#             else:
#                 k_x = 2*pi*(k_i-N_c)/N_c
#             if k_j <= N_c/2:
#                 k_y = 2*pi/N_c*k_j
#             else:
#                 k_y = 2*pi*(k_j-N_c)/N_c
#             if k_k <= N_c/2:
#                 k_z = 2*pi/N_c*k_k
#             else:
#                 k_z = 2*pi*(k_k-N_c)/N_c
#             kk_z[k_k] = k_z
#             kk_x[k_i] = k_x
#             kk_y[k_j] = k_y

# kx, ky, kz = np.meshgrid(kk_x, kk_x, kk_z, indexing='ij')

# D = ((2 * np.sin(kx/2))**2 + (2 * np.sin(ky/2))**2 + (2 * np.sin(kz/2))**2 + 1e-19) # pull out the denominator
# factor = L / Nc # factor due to the normalization of FFt
# omega_k = - (4 * np.pi * G) / D * factor

# # Inverse FFT for the potential field:
# phi = np.fft.irfftn(omega_k * rho_k)


In [144]:
test = potential_field(density_field(pos))
test_xy = np.zeros((N_c,N_c))
for i in range(N_c):
    for j in range(N_c):
        test_xy[i,j] = test[i,j,0]

print(test.shape)

plt.figure()
plt.imshow(test_xy,origin='lower',extent=[0.0,1.0,0.0,1.0])
plt.xlabel('x')
plt.ylabel('y')
plt.title('potential on x-y plane')
cb = plt.colorbar()
plt.show()

ValueError: operands could not be broadcast together with shapes (128,) (128,128,65) 